In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

23/12/24 20:06:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql.functions import broadcast, split, lit, col, split

In [3]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [4]:
matches = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/matches.csv")
matchDetail =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/match_details.csv")
medals = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/medals.csv")
maps =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/maps.csv")


In [5]:
medal_matches =  spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("/home/iceberg/data/medals_matches_players.csv")

In [6]:
matches.createOrReplaceTempView("matches")
matchDetail.createOrReplaceTempView("match_details")
medals.createOrReplaceTempView("medals")
maps.createOrReplaceTempView("maps")
medal_matches.createOrReplaceTempView("medal_matches")

23/12/24 20:07:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [17]:
spark.sql("""
    SELECT * FROM match_details
        
""")

DataFrame[mapid: string, name: string, description: string]

In [7]:
%%sql

SELECT *
FROM matches limit 5

match_id,mapid,is_team_game,playlist_id,game_variant_id,is_match_over,completion_date,match_duration,game_mode,map_variant_id
11de1a94-8d07-4162-9f5f-d3cc753c811c,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,True,2016-02-22 00:00:00,None,None,None
d3643e71-3e51-43e6-a200-f4a7f306ac12,cb914b9e-f206-11e4-b447-24be05e24f7e,False,d0766624-dbd7-4536-ba39-2d890a6143a9,257a305e-4dd3-41f1-9824-dfe7e8bd59e1,True,2016-02-14 00:00:00,None,None,None
d78d2aae-36e4-48ac-a3b5-6d4d90f90ace,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,True,2016-03-24 00:00:00,None,None,55e5ee2e-88df-4657-b9ae-b6ec7ca64614
b440069e-ec5f-4f51-bdd1-bc0bc7fe1195,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,True,2015-12-23 00:00:00,None,None,ec3eef73-13e3-4d4b-a922-cc195109a842
1dd475fc-ee6b-4e1d-8140-c44d03812076,c93d708f-f206-11e4-a815-24be05e24f7e,True,0e39ead4-383b-4452-bbd4-babb7becd82e,42f97cca-2cb4-497a-a0fd-ceef1ba46bcc,True,2016-04-07 00:00:00,None,None,None


In [9]:
%%sql

SELECT *
FROM match_details limit 5

match_id,player_gamertag,previous_spartan_rank,spartan_rank,previous_total_xp,total_xp,previous_csr_tier,previous_csr_designation,previous_csr,previous_csr_percent_to_next_tier,previous_csr_rank,current_csr_tier,current_csr_designation,current_csr,current_csr_percent_to_next_tier,current_csr_rank,player_rank_on_team,player_finished,player_average_life,player_total_kills,player_total_headshots,player_total_weapon_damage,player_total_shots_landed,player_total_melee_kills,player_total_melee_damage,player_total_assassinations,player_total_ground_pound_kills,player_total_shoulder_bash_kills,player_total_grenade_damage,player_total_power_weapon_damage,player_total_power_weapon_grabs,player_total_deaths,player_total_assists,player_total_grenade_kills,did_win,team_id
71d79b23-4143-4359-a62e-489a27597b23,taterbase,5,5,12537,13383,1,3,0,98,None,2,3,0,26,None,4,False,PT14.81149S,6,4,255.0,28,0,0.0,0,0,0,0.0,0.0,0,13,1,0,1,1
71d79b23-4143-4359-a62e-489a27597b23,SuPeRSaYaInG0D,18,18,131943,132557,2,3,0,2,None,1,3,0,76,None,7,False,PT11.2990845S,7,3,350.58792304992676,49,1,45.0,0,0,0,0.0,0.0,0,18,2,0,0,0
71d79b23-4143-4359-a62e-489a27597b23,EcZachly,21,21,168811,169762,2,5,0,94,None,3,5,0,24,None,3,False,PT19.1357063S,12,12,625.0,43,0,0.0,0,0,0,0.0,0.0,0,10,4,0,1,1
71d79b23-4143-4359-a62e-489a27597b23,johnsnake04,14,14,64073,64639,None,None,None,None,None,None,None,None,None,None,6,False,PT21.1521599S,13,13,605.0,24,0,0.0,0,0,0,0.0,0.0,0,9,2,0,0,0
71d79b23-4143-4359-a62e-489a27597b23,Super Mac Bros,26,26,243425,244430,1,5,0,86,None,2,5,0,8,None,2,False,PT12.8373793S,13,12,595.0,32,1,20.004501342773438,0,0,0,0.0,0.0,0,15,2,0,1,1


In [21]:
%%sql

SELECT *
FROM maps LIMIT 5

mapid,name,description
c93d708f-f206-11e4-a815-24be05e24f7e,Urban,Andesia was the crucible for countless heroes and villains caught in the throes of seething rebellion and righteous excess.
cb251c51-f206-11e4-8541-24be05e24f7e,Raid on Apex 7,This unbroken ring is a symbol of the discovery that shook the galaxy and changed the course of both human and Covenant destiny.
c854e54f-f206-11e4-bddc-24be05e24f7e,March on Stormbreak,None
c8d69870-f206-11e4-b477-24be05e24f7e,Escape from A.R.C.,Scientists flocked to this Forerunner excavation in search of new beginnings. What they unearthed will lead to their inevitable end.
73ed1fd0-45e5-4bb9-ab6a-d2852c04ea7d,Osiris,None


In [22]:
%%sql

SELECT *
FROM medals LIMIT 5

medal_id,sprite_uri,sprite_left,sprite_top,sprite_sheet_width,sprite_sheet_height,sprite_width,sprite_height,classification,description,name,difficulty
2315448068,None,None,None,None,None,None,None,None,None,None,None
3565441934,None,None,None,None,None,None,None,None,None,None,None
4162659350,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,750,750,74,74,1125,899,Breakout,Kill the last enemy within the last 10 seconds of a round.,Buzzer Beater,45
1573153198,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,0,300,74,74,1125,899,Breakout,Survive a one-on-one encounter.,Vanquisher,30
298813630,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,0,825,74,74,1125,899,Style,Kill an enemy with Spartan Charge.,Spartan Charge,135


In [23]:
%%sql

SELECT *
FROM medal_matches LIMIT 5

match_id,player_gamertag,medal_id,count
009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,3261908037,7
009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,824733727,2
009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,2078758684,2
009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,2782465081,2
9169d1a3-955c-4ea9-a9a4-6d57da097660,EcZachly,3001183151,1


In [ ]:
#matches, medal_matches, match_details.

In [7]:
#Joining the matches, match_details, medal_matches table using match_id 
bucketedValues = matches.alias("m").join(matchDetail.alias("md"), col("m.match_id") == col("md.match_id")) \
     .join(medal_matches.alias("mm"), col("m.match_id") == col("mm.match_id"))

In [8]:
#creating DDL for the joined table for bucket join
spark.sql("""DROP TABLE IF EXISTS bootcamp.matchdetailsmedals""")
bucketedDDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.matchdetailsmedals (
    match_id STRING,
    mapid STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER,
     medal_id BIGINT,
     count INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """

spark.sql(bucketedDDL)

DataFrame[]

In [9]:
#writing to matches, match_details, medals bucket joined tables
bucketedValues.select(
     col("m.match_id"), col("mapid"),col("is_team_game"), col("playlist_id"), col("completion_date"), \
    col("mm.player_gamertag"), col("player_total_kills"), col("player_total_deaths"), \
     col("medal_id"), col("count")
     ) \
     .write.mode("append")  \
     .bucketBy(16, "match_id").saveAsTable("bootcamp.matchdetailsmedals")

In [12]:
explicitBroadcast = bucketedValues.alias("bv").join(broadcast(maps).alias("mp"), col("bv.mapid") == col("mp.mapid")) \
                .join(broadcast(medals).alias("md"), col("bv.medal_id") == col("md.medal_id"))

In [ ]:
#creating using DDL a table to add the joined table details
%%sql

DROP TABLE IF EXISTS bootcamp.events

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_type STRING,
    os_type STRING,
    device_type STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));

In [ ]:
#insert the values of the join to the table
explicitBroadcast.write.mode("overwrite").insertInto("bootcamp.medal_map_joined")

In [ ]:
#which player has the highest average kills per game? 

In [21]:
%%sql

SELECT *
from bootcamp.matchdetailsmedals 
limit 5



match_id,mapid,is_team_game,playlist_id,completion_date,player_gamertag,player_total_kills,player_total_deaths,medal_id,count
0192fac5-42a7-444f-aede-ed6f70aba2ec,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,2016-05-03 00:00:00,VwS Impressive,8,5,3261908037,7
0192fac5-42a7-444f-aede-ed6f70aba2ec,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,2016-05-03 00:00:00,VwS Impressive,8,5,2078758684,1
0192fac5-42a7-444f-aede-ed6f70aba2ec,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,2016-05-03 00:00:00,VwS Impressive,8,5,298813630,1
0192fac5-42a7-444f-aede-ed6f70aba2ec,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,2016-05-03 00:00:00,VwS Impressive,8,5,2782465081,1
0192fac5-42a7-444f-aede-ed6f70aba2ec,c7edbf0f-f206-11e4-aa52-24be05e24f7e,True,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,2016-05-03 00:00:00,RD The Hurban,8,5,3261908037,1


In [31]:
%%sql
select distinct player_gamertag, match_id, 
AVG(player_total_kills) OVER(partition by player_gamertag order by match_id) as avg_kills
from bootcamp.matchdetailsmedals
order by 3 DESC

player_gamertag,match_id,avg_kills
GsFurreal,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
zombiesrhunters,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
gimpinator14,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
WhiteMountainDC,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
hibblesnbitz,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
XoptimusprideX,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
taurenmonk,acf0e47e-20ac-4b12-b292-591d4b3a3df9,56.0
ForbiddenArtist,3ec9c9c9-4924-45e4-a478-04c12a7da912,54.75
BaronTheFox318,3ec9c9c9-4924-45e4-a478-04c12a7da912,54.75
ProphetofDoom01,3ec9c9c9-4924-45e4-a478-04c12a7da912,54.75


In [ ]:
#which playlist has received the most plays? (query_4b)


In [34]:
%%sql

select playlist_id, COUNT(match_id) 
from bootcamp.matchdetailsmedals
GROUP BY 1
order by 2 DESC

playlist_id,count(match_id)
f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1565529
780cc101-005c-4fca-8ce7-6f36d7156ffe,1116002
0bcf2be1-3168-4e42-9fb5-3551d7dbce77,1015496
c98949ae-60a8-43dc-85d7-0feb0b92e719,824932
2323b76a-db98-4e03-aa37-e171cfbdd1a4,692342
892189e9-d712-4bdb-afa7-1ccab43fbed4,667670
f27a65eb-2d11-4965-aa9c-daa088fa5c9c,167498
355dc154-9809-4edb-8ed4-fff910c6ae9c,140006
d0766624-dbd7-4536-ba39-2d890a6143a9,138470
bc0f8ad6-31e6-4a18-87d9-ad5a2dbc8212,111073


In [ ]:
#which map was played the most?

In [36]:
%%sql

select distinct mapid, COUNT(mapid) 
from bootcamp.matchdetailsmedals
GROUP BY 1
order by 2 DESC

mapid,count(mapid)
c74c9d0f-f206-11e4-8330-24be05e24f7e,1445545
c7edbf0f-f206-11e4-aa52-24be05e24f7e,1435048
c7805740-f206-11e4-982c-24be05e24f7e,953278
cdb934b0-f206-11e4-8810-24be05e24f7e,396305
cb914b9e-f206-11e4-b447-24be05e24f7e,309045
ce1dc2de-f206-11e4-a646-24be05e24f7e,299736
cebd854f-f206-11e4-b46e-24be05e24f7e,298891
caacb800-f206-11e4-81ab-24be05e24f7e,291540
cd844200-f206-11e4-9393-24be05e24f7e,261162
cc040aa1-f206-11e4-a3e0-24be05e24f7e,256966


In [ ]:
#on which map do players receive the highest number of Killing Spree medals?

In [37]:
%%sql

select distinct mapid, COUNT(player_total_kills) 
from bootcamp.matchdetailsmedals
GROUP BY 1
order by 2 DESC

23/12/24 21:47:21 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

mapid,count(player_total_kills)
c74c9d0f-f206-11e4-8330-24be05e24f7e,1445545
c7edbf0f-f206-11e4-aa52-24be05e24f7e,1435048
c7805740-f206-11e4-982c-24be05e24f7e,953278
cdb934b0-f206-11e4-8810-24be05e24f7e,396305
cb914b9e-f206-11e4-b447-24be05e24f7e,309045
ce1dc2de-f206-11e4-a646-24be05e24f7e,299736
cebd854f-f206-11e4-b46e-24be05e24f7e,298891
caacb800-f206-11e4-81ab-24be05e24f7e,291540
cd844200-f206-11e4-9393-24be05e24f7e,261162
cc040aa1-f206-11e4-a3e0-24be05e24f7e,256966


23/12/24 21:47:31 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [38]:
first_sort_df = bucketedValues.sortWithinPartitions(col("match_id"), col("completion_date"), col("mapid"))



ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: functions does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
second_sort_df = bucketedValues.repartition(10, col("completion_date")) \
        .sortWithinPartitions(col("match_id")) \
        .withColumn("completion_date", col("completion_date").cast("timestamp")) \

In [ ]:
third_sort_df = bucketedValues.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \